# Basic RNN without TF RNN operations

An unrolled rnn with 5 neurons assuming 2 time steps. Input vector of size 3

In [1]:
n_inputs = 3
n_neurons = 5

In [2]:
import tensorflow as tf
X0 = tf.placeholder(dtype=tf.float32, shape=(None,n_inputs))
X1 = tf.placeholder(dtype=tf.float32, shape=(None,n_inputs))

Wx = tf.Variable(tf.random_normal(shape=[n_inputs,n_neurons],
                                 dtype=tf.float32))
Wy = tf.Variable(tf.random_normal(shape=[n_neurons,n_neurons],
                                 dtype=tf.float32))
b = tf.Variable(tf.random_normal(shape=[1,n_neurons],
                                dtype=tf.float32))

Y0 = tf.tanh(tf.matmul(X0,Wx) + b)
Y1 = tf.tanh(tf.matmul(X1,Wx) + tf.matmul(Y0,Wy) + b)

init = tf.global_variables_initializer()

In [3]:
import numpy as np
X0_batch = np.array([[0, 1, 2], [3, 4, 5], [6, 7, 8], [9, 0, 1]])
X1_batch = np.array([[9, 8, 7], [0, 0, 0], [6, 5, 4], [3, 2, 1]])

with tf.Session() as sess:
    init.run()
    
    Y0_val, Y1_val = sess.run([Y0,Y1], feed_dict={X0:X0_batch,
                                          X1:X1_batch})

In [4]:
print (Y0_val)

[[ 0.9069021   0.95094186  0.9999823  -0.99908364  0.9762498 ]
 [ 0.9993543   0.9925049   1.         -0.7203413   0.9999986 ]
 [ 0.9999958   0.9988751   1.          0.9659054   1.        ]
 [ 0.99292046  0.99984086 -0.99999917  0.99997395 -0.77904946]]


In [5]:
print (Y1_val)

[[ 0.99997604 -0.9875312   1.          0.9999999   1.        ]
 [-0.7701138  -0.9989653  -0.9385692  -0.9502374  -0.7617843 ]
 [ 0.9764096  -0.83620816  0.99999845  0.9999039   0.9999998 ]
 [ 0.07642302  0.99994564  0.91755545  0.9977068   0.99500686]]


# Same graph using TF RNN operation

static_rnn creates an unrolled RNN by chaining cells together.

In [6]:
"""
X0 = tf.placeholder(dtype=tf.float32,shape=[None,n_inputs])
X1 = tf.placeholder(dtype=tf.float32,shape=[None,n_inputs])

basic_cell = tf.contrib.rnn.BasicRNNCell(num_units=n_neurons)
output_seqs, states = tf.contrib.rnn.static_rnn(cell=basic_cell,
                                               inputs=[X0,X1],
                                               dtype=tf.float32)

Y0_rnn,Y1_rnn = output_seqs
"""

'\nX0 = tf.placeholder(dtype=tf.float32,shape=[None,n_inputs])\nX1 = tf.placeholder(dtype=tf.float32,shape=[None,n_inputs])\n\nbasic_cell = tf.contrib.rnn.BasicRNNCell(num_units=n_neurons)\noutput_seqs, states = tf.contrib.rnn.static_rnn(cell=basic_cell,\n                                               inputs=[X0,X1],\n                                               dtype=tf.float32)\n\nY0_rnn,Y1_rnn = output_seqs\n'

# Training to Predict Time Series

In [7]:
n_neurons = 100
n_steps = 20
n_inputs = 1
n_outputs = 1

In [8]:
X = tf.placeholder(dtype=tf.float32,shape=[None,n_steps,n_inputs])
y = tf.placeholder(dtype=tf.float32,shape=[None,n_steps,n_outputs])

rnn_cell = tf.contrib.rnn.BasicRNNCell(num_units=100,activation=tf.nn.relu)
outputs_rnn, states_rnn = tf.nn.dynamic_rnn(rnn_cell,X,dtype=tf.float32)


In [10]:
cell = tf.contrib.rnn.OutputProjectionWrapper(
            tf.contrib.rnn.BasicRNNCell(num_units=n_neurons,
                                       activation=tf.nn.relu),
            output_size = n_outputs)

In [11]:
learning_rate  = 0.001

loss = tf.reduce_mean(tf.square(outputs_rnn - y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(loss)

init = tf.global_variables_initializer()


In [ ]:
n_iterations = 1500
batch_size=50

with tf.Session() as sess:
    init.run()
    
    for iteration in range(n_iterations):
        X_batch, y_batch = [...] #Fetch time series data batch
        X_batch = X_batch.reshape((-1,n_steps,n_inputs))
        sess.run(training_op, feed_dict={X:X_batch, y:y_batch})
        
        if iteration % 100 == 0:
            mse = loss.eval(feed_dict={X:X_batch, y:y_batch})
            print (iteration, "\tMSE:", mse)
        